In [132]:
import requests
import json
from datetime import datetime, timedelta
from dateutil import tz
import re
import csv

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')
key = ''
lat = '40.730610'
lon = '-73.935242'

def scrape_weather(time=datetime(2013, 1, 1, 0, 0, 0, tzinfo=tz.gettz('America/New_York')), count=900):
    with open('weather.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for day_count in range(count):
            print(time.strftime("%Y-%m-%dT%H:%M:%S%z"))
            url = 'https://api.darksky.net/forecast/{}/{},{},{}?exclude=currently,flags'.format(key, lat, lon, 
                                                                                                time.strftime("%Y-%m-%dT%H:%M:%S%z")
                                                                                               )
            r = requests.get(url, 'json')
            for hour in scrape_day(json.loads(r.content)):
                writer.writerow(hour)
            time += timedelta(days=1)

In [133]:
def scrape_day(day):
    sunrise = day['daily']['data'][0]['sunriseTime']
    sunset = day['daily']['data'][0]['sunsetTime']
    
    for ele in day['hourly']['data']:
        hourly = []
        hourly.append(
            datetime.utcfromtimestamp(ele['time']).replace(tzinfo=from_zone).astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S')
        )
        if sunrise <= ele['time'] <= sunset:
            hourly.append(1)
        else:
            hourly.append(0)
        hourly.append(re.sub('(-night)|(-day)', '', ele['icon'].lower()))
        hourly.append(ele['precipProbability'])
        hourly.append(ele['apparentTemperature'])
        hourly.append(ele['humidity'])
        hourly.append(ele['windSpeed'])
        yield hourly

scrape_weather(count=2)

2013-01-01T00:00:00-0500
2013-01-02T00:00:00-0500
